In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [14]:
df_o = pd.read_csv('beta_origin.csv', low_memory=False).drop('Unnamed: 0', axis=1).dropna(subset=['origin']).sort_values(by='origin_id')
df_l = pd.read_csv('location.csv', low_memory=False).drop('id', axis=1)

In [15]:
df_l['origin_id'] = np.nan
df_l['orig_cert'] = np.nan
df_o['origin_id'] = df_o['origin_id'].astype(int)

In [16]:
# df_l[df_l.duplicated(subset=['place'], keep=False)].to_csv('duplicated_place_names.csv')

In [17]:
for index, row in df_l.iterrows():
    match = df_o[df_o['origin'] == row['place']]

    # We know each place appears at most once in df_o
    if not match.empty:
        match_row = match.iloc[0]

        df_l.at[index, 'origin_id'] = int(match_row['origin_id'])
        df_l.at[index, 'orig_cert'] = match_row['orig_cert']

        for oc in ['orig_class_1', 'orig_class_2', 'orig_class_3']:
            if (pd.isna(row[oc]) or row[oc] == '/') and pd.notna(match_row[oc]):
                df_l.at[index, oc] = match_row[oc]

In [18]:
for place in tqdm(set(df_o['origin'])):
    if place not in set(df_l['place']):
        match_row = df_o[df_o['origin'] == place].iloc[0]

        new_row = {
            'place': place,
            'orig_cert': match_row['orig_cert'],
            'origin_id': int(match_row['origin_id']),
            'orig_class_1': match_row['orig_class_1'],
            'orig_class_2': match_row['orig_class_2'],
            'orig_class_3': match_row['orig_class_3'],
        }

        df_l = pd.concat([df_l, pd.DataFrame([new_row])], ignore_index=True)

100%|██████████| 8950/8950 [00:39<00:00, 224.55it/s]


In [19]:
df_l.sort_values(by='origin_id').to_csv('combined_loc.csv')